<a href="https://colab.research.google.com/github/irfanbykara/Machine-Learning-Notebooks/blob/main/ResNet_Implementation_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here is the implementation of Resnet-34 from scratch...


In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler



In [ ]:
# Device agnostic
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
def data_loader(data_dir,
                batch_size,
                random_seed=42,
                valid_size=0.1,
                shuffle=True,
                test=False):
  
    normalize = transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010],
    )

    # define transforms
    transform = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            normalize,
    ])

    if test:
        dataset = datasets.CIFAR10(
          root=data_dir, train=False,
          download=True, transform=transform,
        )

        data_loader = torch.utils.data.DataLoader(
            dataset, batch_size=batch_size, shuffle=shuffle
        )

        return data_loader

    # load the dataset
    train_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=transform,
    )

    valid_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=transform,
    )

    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(42)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, sampler=train_sampler)
 
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=batch_size, sampler=valid_sampler)

    return (train_loader, valid_loader)


# CIFAR10 dataset 
train_loader, valid_loader = data_loader(data_dir='./data',
                                         batch_size=64)

test_loader = data_loader(data_dir='./data',
                              batch_size=64,
                              test=True)


100%|██████████| 170498071/170498071 [00:02<00:00, 84777688.33it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
class ResidualBlock(nn.Module):
  def __init__(self,in_channels,out_channels,stride=1,downsample=None):
    super(ResidualBlock,self).__init__()
    self.conv1 = nn.Sequential(
        nn.Conv2d(in_channels,out_channels,kernel_size=3,stride=stride,padding=1),
          nn.BatchNorm2d(out_channels),
          nn.ReLU()
          )
    self.conv2 = nn.Sequential(
      nn.Conv2d(out_channels,out_channels,kernel_size=3,stride=1,padding=1),
      nn.BatchNorm2d(out_channels)
    )
    self.relu = nn.ReLU()
    self.out_channels = out_channels
    self.downsample = downsample
  
  def forward(self,x):
    residual = x
    out = self.conv1(x)
    out = self.conv2(out)
    if self.downsample:
      residual = self.downsample(x)
    # print(out.shape,residual.shape)
    out += residual
    out = self.relu(out)
    return out
    

In [ ]:
class ResNet(nn.Module):
  def __init__(self,block,layers,num_classes):
    super(ResNet,self).__init__()
    self.in_channels = 64
    self.conv1 = nn.Sequential(
        nn.Conv2d(3,64,kernel_size=7,stride=2,padding=3),
        nn.BatchNorm2d(64),
        nn.ReLU()
    )
    self.max_pool = nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
    self.layer0 = self._get_layer(block,64,layers[0],stride=1)
    self.layer1 = self._get_layer(block, 128, layers[1], stride = 2)
    self.layer2 = self._get_layer(block, 256, layers[2], stride = 2)
    self.layer3 = self._get_layer(block, 512, layers[3], stride = 2)
    self.avgpool = nn.AvgPool2d(7, stride=1)
    self.fc = nn.Linear(512, num_classes)

  


  def _get_layer(self,block,channels,blocks,stride=1):
    downsample = None
    if stride != 1 or self.in_channels != channels:
      downsample = nn.Sequential(
          nn.Conv2d(self.in_channels,channels,kernel_size = 1,stride=stride,),
          nn.BatchNorm2d(channels)
      )
    layers = []
    layers.append(block(self.in_channels,channels,stride,downsample))
    self.in_channels = channels
    for i in range(1, blocks):
      layers.append(block(self.in_channels, channels))

    return nn.Sequential(*layers)

   
  def forward(self, x):
      x = self.conv1(x)
      x = self.max_pool(x)
      x = self.layer0(x)
      x = self.layer1(x)
      x = self.layer2(x)
      x = self.layer3(x)

      x = self.avgpool(x)
      x = x.view(x.size(0), -1)
      x = self.fc(x)

      return x


In [ ]:
num_classes = 10
num_epochs = 20
batch_size = 16
learning_rate = 0.01

model = ResNet(ResidualBlock, [3, 4, 6, 3],num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.001, momentum = 0.9)  



In [ ]:
len(train_loader)

704

In [ ]:
import gc
total_step = len(train_loader)

train_acc = []
valid_acc = []
for epoch in range(num_epochs):
  correct_train = 0 
  total_train = 0
  for i,(images,labels) in enumerate(train_loader):
    
    images = images.to(device)
    labels = labels.to(device)
    output = model(images)
    _, predicted = torch.max(output.data, 1)
    total_train += labels.size(0)
    correct_train += (predicted == labels).sum().item()
    loss = criterion(output,labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    del images,labels,output
    torch.cuda.empty_cache()
    gc.collect()

  print ('Epoch [{}/{}], Loss: {:.4f}' 
              .format(epoch+1, num_epochs, loss.item()))
  print('Accuracy of the network on the {} train images: {} %'.format(5000, 100 * correct_train / total_train)) 
  train_acc.append(correct_train / total_train)

  

  with torch.no_grad():
    correct_valid = 0
    total_valid = 0
    for images, labels in valid_loader:
      images = images.to(device)
      labels = labels.to(device)

      output = model(images)
      _, predicted = torch.max(output.data, 1)
      total_valid += labels.size(0)
      correct_valid += (predicted == labels).sum().item()
      del images, labels, output
  
    print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct_valid / total_valid)) 
    valid_acc.append(correct_valid / total_valid)



Epoch [1/20], Loss: 1.0085
Accuracy of the network on the 5000 train images: 77.09333333333333 %
Accuracy of the network on the 5000 validation images: 76.2 %
Epoch [2/20], Loss: 0.6771
Accuracy of the network on the 5000 train images: 81.31555555555556 %
Accuracy of the network on the 5000 validation images: 80.54 %
Epoch [3/20], Loss: 1.1754
Accuracy of the network on the 5000 train images: 84.43111111111111 %
Accuracy of the network on the 5000 validation images: 81.56 %
Epoch [4/20], Loss: 0.0592
Accuracy of the network on the 5000 train images: 86.59333333333333 %
Accuracy of the network on the 5000 validation images: 81.58 %
Epoch [5/20], Loss: 0.0370
Accuracy of the network on the 5000 train images: 88.81777777777778 %
Accuracy of the network on the 5000 validation images: 83.16 %
Epoch [6/20], Loss: 0.7672
Accuracy of the network on the 5000 train images: 90.51555555555555 %
Accuracy of the network on the 5000 validation images: 82.82 %
Epoch [7/20], Loss: 0.1468
Accuracy of th